In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'WAT'
start_date = '2020-01-01'
end_date = '2024-01-01'
1
# Hyperparameters
n_steps = 2
window_size = 5
step_size = 2
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.00005
train_ratio = 0.9
n_units = 16
batch_size = 4

# If you change features or other logic, do so below
if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")


def load_full_data_helper(ticker, start_date, end_date, window_size):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Next day prediction setup
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    # Simple features
    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates


def load_full_data(ticker, start_date, end_date, window_size):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates


def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)

    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)


def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    # We'll compile after loading weights if needed
    return model


def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    global train_ratio
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape


def rolling_training_early_stopping(ticker, window_size, step_size, n_steps):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Early Stopping...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for {ticker}. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    # Just run with the current top-level defined parameters:
    rolling_training_early_stopping(ticker, window_size, step_size, n_steps)


[*********************100%%**********************]  1 of 1 completed
[INFO] Training initial window with Early Stopping...
[RESULT] Initial window validation MAPE: 0.14%

[INFO] Iteration 1: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.05%
[INFO] Predicted last sample of window: Actual=228.82, Pred=228.94, MAPE=0.05%

[INFO] Iteration 2: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.56%
[INFO] Predicted last sample of window: Actual=234.36, Pred=230.71, MAPE=1.56%

[INFO] Iteration 3: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.04%
[INFO] Predicted last sample of window: Actual=239.60, Pred=234.71, MAPE=2.04%

[INFO] Iteration 4: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.78%
[INFO] Predicted last sample of window: Actual=244.09, Pred=239.75, MAPE=1.78%

[INFO] Iteration 5: training on new window with Early Stopping...
[RESULT] Window validat

[RESULT] Window validation MAPE: 1.03%
[INFO] Predicted last sample of window: Actual=178.46, Pred=180.30, MAPE=1.03%

[INFO] Iteration 45: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.84%
[INFO] Predicted last sample of window: Actual=183.99, Pred=180.61, MAPE=1.84%

[INFO] Iteration 46: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.00%
[INFO] Predicted last sample of window: Actual=185.26, Pred=185.26, MAPE=0.00%

[INFO] Iteration 47: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.97%
[INFO] Predicted last sample of window: Actual=193.08, Pred=185.41, MAPE=3.97%

[INFO] Iteration 48: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.66%
[INFO] Predicted last sample of window: Actual=197.96, Pred=194.67, MAPE=1.66%

[INFO] Iteration 49: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.35%
[INFO] Predicted last sample of

[RESULT] Window validation MAPE: 1.87%
[INFO] Predicted last sample of window: Actual=190.60, Pred=194.17, MAPE=1.87%

[INFO] Iteration 90: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.29%
[INFO] Predicted last sample of window: Actual=191.06, Pred=190.51, MAPE=0.29%

[INFO] Iteration 91: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.18%
[INFO] Predicted last sample of window: Actual=193.39, Pred=191.10, MAPE=1.18%

[INFO] Iteration 92: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.01%
[INFO] Predicted last sample of window: Actual=195.68, Pred=193.70, MAPE=1.01%

[INFO] Iteration 93: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.47%
[INFO] Predicted last sample of window: Actual=198.65, Pred=195.74, MAPE=1.47%

[INFO] Iteration 94: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.26%
[INFO] Predicted last sample of

[RESULT] Window validation MAPE: 0.79%
[INFO] Predicted last sample of window: Actual=264.67, Pred=266.75, MAPE=0.79%

[INFO] Iteration 134: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 8.38%
[INFO] Predicted last sample of window: Actual=291.31, Pred=266.90, MAPE=8.38%

[INFO] Iteration 135: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.87%
[INFO] Predicted last sample of window: Actual=286.45, Pred=283.95, MAPE=0.87%

[INFO] Iteration 136: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.14%
[INFO] Predicted last sample of window: Actual=284.50, Pred=284.90, MAPE=0.14%

[INFO] Iteration 137: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.01%
[INFO] Predicted last sample of window: Actual=284.02, Pred=284.04, MAPE=0.01%

[INFO] Iteration 138: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.02%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 0.28%
[INFO] Predicted last sample of window: Actual=318.71, Pred=317.81, MAPE=0.28%

[INFO] Iteration 178: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.59%
[INFO] Predicted last sample of window: Actual=328.00, Pred=319.50, MAPE=2.59%

[INFO] Iteration 179: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.41%
[INFO] Predicted last sample of window: Actual=336.58, Pred=328.48, MAPE=2.41%

[INFO] Iteration 180: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.62%
[INFO] Predicted last sample of window: Actual=341.80, Pred=336.26, MAPE=1.62%

[INFO] Iteration 181: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.20%
[INFO] Predicted last sample of window: Actual=346.56, Pred=342.39, MAPE=1.20%

[INFO] Iteration 182: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.09%
[INFO] Predicted last samp

...

[RESULT] Window validation MAPE: 3.18%
[INFO] Predicted last sample of window: Actual=321.44, Pred=311.21, MAPE=3.18%

[INFO] Iteration 310: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.95%
[INFO] Predicted last sample of window: Actual=330.24, Pred=323.78, MAPE=1.95%

[INFO] Iteration 311: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.03%
[INFO] Predicted last sample of window: Actual=329.60, Pred=329.50, MAPE=0.03%

[INFO] Iteration 312: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.99%
[INFO] Predicted last sample of window: Actual=336.89, Pred=330.19, MAPE=1.99%

[INFO] Iteration 313: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.23%
[INFO] Predicted last sample of window: Actual=342.27, Pred=346.48, MAPE=1.23%

[INFO] Iteration 314: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.08%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 0.37%
[INFO] Predicted last sample of window: Actual=300.96, Pred=299.86, MAPE=0.37%

[INFO] Iteration 354: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.61%
[INFO] Predicted last sample of window: Actual=299.17, Pred=301.00, MAPE=0.61%

[INFO] Iteration 355: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.12%
[INFO] Predicted last sample of window: Actual=295.74, Pred=296.09, MAPE=0.12%

[INFO] Iteration 356: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.67%
[INFO] Predicted last sample of window: Actual=305.07, Pred=296.91, MAPE=2.67%

[INFO] Iteration 357: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.97%
[INFO] Predicted last sample of window: Actual=312.56, Pred=306.42, MAPE=1.97%

[INFO] Iteration 358: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 6.40%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 4.04%
[INFO] Predicted last sample of window: Actual=309.13, Pred=321.61, MAPE=4.04%

[INFO] Iteration 398: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.02%
[INFO] Predicted last sample of window: Actual=309.35, Pred=309.42, MAPE=0.02%

[INFO] Iteration 399: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.67%
[INFO] Predicted last sample of window: Actual=306.34, Pred=308.39, MAPE=0.67%

[INFO] Iteration 400: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.70%
[INFO] Predicted last sample of window: Actual=301.85, Pred=306.99, MAPE=1.70%

[INFO] Iteration 401: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.32%
[INFO] Predicted last sample of window: Actual=298.97, Pred=305.91, MAPE=2.32%

[INFO] Iteration 402: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.21%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 1.21%
[INFO] Predicted last sample of window: Actual=269.26, Pred=265.99, MAPE=1.21%

[INFO] Iteration 442: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.21%
[INFO] Predicted last sample of window: Actual=269.83, Pred=269.28, MAPE=0.21%

[INFO] Iteration 443: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.31%
[INFO] Predicted last sample of window: Actual=280.08, Pred=270.82, MAPE=3.31%

[INFO] Iteration 444: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.03%
[INFO] Predicted last sample of window: Actual=288.72, Pred=279.99, MAPE=3.03%

[INFO] Iteration 445: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.67%
[INFO] Predicted last sample of window: Actual=286.54, Pred=288.45, MAPE=0.67%

[INFO] Iteration 446: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.58%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 3.83%
[INFO] Predicted last sample of window: Actual=266.94, Pred=256.73, MAPE=3.83%

[INFO] Iteration 486: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.19%
[INFO] Predicted last sample of window: Actual=261.88, Pred=262.39, MAPE=0.19%

[INFO] Iteration 487: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.23%
[INFO] Predicted last sample of window: Actual=274.65, Pred=265.78, MAPE=3.23%

[INFO] Iteration 488: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.91%
[INFO] Predicted last sample of window: Actual=280.39, Pred=275.05, MAPE=1.91%

[INFO] Iteration 489: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.16%
[INFO] Predicted last sample of window: Actual=275.95, Pred=276.39, MAPE=0.16%

[INFO] Iteration 490: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.24%
[INFO] Predicted last samp